# Optic Disk Segmentation Eval

In [1]:
%load_ext autoreload
%autoreload 2

In [21]:

from fundus_odmac_toolkit.models.segmentation import segment
from fundus_odmac_toolkit.models.hf_hub import list_models
from fundus_data_toolkit.functional import open_image
from fundus_data_toolkit.utils.visualization import plot_image_and_mask
import cv2
import torch
from pathlib import Path
import torchmetrics
from tqdm.notebook import tqdm

import pandas as pd

In [3]:
all_models = list_models()

Architecture |  Encoder |  Variants
unet seresnet50  (1 variants)
unet maxvit_tiny_tf_512  (1 variants)
unet maxvit_base_tf_512  (1 variants)
unet mobilevitv2_100  (1 variants)
unetplusplus seresnet50  (1 variants)
unet mobilenetv3_small_050  (1 variants)
unetplusplus mobilenetv3_small_050  (1 variants)
unet maxvit_small_tf_512  (1 variants)


In [4]:

root_img = Path('/home/clement/Documents/data/IDRID/A. Segmentation/1. Original Images/b. Testing Set/')
root_masks = Path('/home/clement/Documents/data/IDRID/A. Segmentation/2. All Segmentation Groundtruths/b. Testing Set/5. Optic Disc/')

all_files_names = [f.stem for f in root_img.glob('*.jpg')]
def infer(arch, encoder):
    jaccardIndex = torchmetrics.JaccardIndex(task='binary').cuda()
    for file_name in tqdm(all_files_names):
        img_name = (root_img / file_name).with_suffix('.jpg')
        mask_name = (root_masks / (file_name+'_OD')).with_suffix('.tif')
        mask = cv2.imread(str(mask_name), cv2.IMREAD_GRAYSCALE)
        
        mask = mask > 0
        mask = torch.tensor(mask).cuda().float()
        
        
        image = open_image(img_name)
        pred = segment(image, use_tta=False, autofit_resolution=True, reverse_autofit=True, arch=arch, encoder=encoder)
        pred = torch.argmax(pred, dim=0)
        pred = (pred == 1).long()
        jaccardIndex.update(pred, mask)
    print(f"Model: {arch}-{encoder}, Jaccard Index: {jaccardIndex.compute().item():.2%}")

for arch, encoder in all_models:
    infer(arch, encoder)
  

  0%|          | 0/27 [00:00<?, ?it/s]

Model: unet-seresnet50, Jaccard Index: 92.03%


  0%|          | 0/27 [00:00<?, ?it/s]

Model: unet-maxvit_tiny_tf_512, Jaccard Index: 92.27%


  0%|          | 0/27 [00:00<?, ?it/s]

...

Model: unet-mobilenetv3_small_050, Jaccard Index: 89.13%


  0%|          | 0/27 [00:00<?, ?it/s]

Model: unetplusplus-mobilenetv3_small_050, Jaccard Index: 85.86%


  0%|          | 0/27 [00:00<?, ?it/s]

Model: unet-maxvit_small_tf_512, Jaccard Index: 92.84%


# Optic Disk Localization

In [19]:
root = Path('/home/clement/Documents/data/IDRID/C. Localization/1. Original Images/b. Testing Set/')
solution_filepath = '/home/clement/Documents/data/IDRID/C. Localization/2. Groundtruths/1. Optic Disc Center Location/b. IDRiD_OD_Center_Testing Set_Markups.csv'

all_files = list(root.glob('*.jpg'))
gt = pd.read_csv(solution_filepath)

def inferod_loc(arch, encoder, tta=True):
    all_distances = []
    for file in tqdm(all_files):
        filename = file.stem
        image = open_image(file)
        pred = segment(image, use_tta=tta, autofit_resolution=True, reverse_autofit=True, arch=arch, encoder=encoder)
        pred = torch.argmax(pred, dim=0)
        pred = (pred == 1).long()
        y_center, x_center = pred.nonzero().float().mean(dim=0).tolist()
        if y_center==0 and x_center==0:
            continue
        
        gt_x_center, gt_y_center = gt[gt['Image No'] == filename][['X- Coordinate', 'Y - Coordinate']].values[0]
        
        distance = ((x_center - gt_x_center)**2 + (y_center - gt_y_center)**2)**0.5
        all_distances.append(distance)
    print(f"Model: {arch}-{encoder}, Mean distance: {sum(all_distances)/len(all_distances):.2f}")
        

for arch, encoder in all_models:
    inferod_loc(arch, encoder, tta=True)

  0%|          | 0/103 [00:00<?, ?it/s]

Model: unet-seresnet50, Mean distance: 25.73


  0%|          | 0/103 [00:00<?, ?it/s]

Model: unet-maxvit_tiny_tf_512, Mean distance: 30.17


  0%|          | 0/103 [00:00<?, ?it/s]

...

Model: unet-mobilenetv3_small_050, Mean distance: 34.72


  0%|          | 0/103 [00:00<?, ?it/s]

Model: unetplusplus-mobilenetv3_small_050, Mean distance: 42.44


  0%|          | 0/103 [00:00<?, ?it/s]

Model: unet-maxvit_small_tf_512, Mean distance: 29.07


# Fovea localization

In [44]:
import math



solution_filepath_fovea = '/home/clement/Documents/data/IDRID/C. Localization/2. Groundtruths/2. Fovea Center Location/IDRiD_Fovea_Center_Testing Set_Markups.csv'
gt_fovea = pd.read_csv(solution_filepath_fovea)
def inferfovea_loc(arch, encoder, tta=True):
    all_distances = []
    for file in tqdm(all_files):
        filename = file.stem
        image = open_image(file)
        pred = segment(image, use_tta=tta, autofit_resolution=True, reverse_autofit=True, arch=arch, encoder=encoder)
        pred = torch.argmax(pred, dim=0)
        pred = (pred == 2).long()
        y_center, x_center = pred.nonzero().float().mean(dim=0).tolist()
        
        gt_x_center, gt_y_center = gt_fovea[gt_fovea['Image No'] == filename][['X- Coordinate', 'Y - Coordinate']].values[0]
        distance = ((x_center - gt_x_center)**2 + (y_center - gt_y_center)**2)**0.5
        if math.isnan(distance):
            print('Macula was not detected!')
            continue
        all_distances.append(distance)
    print(f"Model: {arch}-{encoder}, Mean distance: {sum(all_distances)/len(all_distances):.2f}") 


inferfovea_loc('unet', 'maxvit_small_tf_512', tta=True)

  0%|          | 0/103 [00:00<?, ?it/s]

Macula was not detected!
Macula was not detected!
Macula was not detected!
Macula was not detected!
Macula was not detected!
Macula was not detected!
Macula was not detected!
Macula was not detected!
Model: unet-maxvit_small_tf_512, Mean distance: 48.12
